# Filter projected genes 

There are two different sets of gene set. One for the assemblies in pseudomolecules (PGSBv2: ```arinalrfor, chinese, jagger, julius, lancer, landmark, mace, norin61, spelta, stanley, sy_mattis```) and in scaffolds (EI: ```cadenza, claire, paragon, robigus, weebil```). 

## EI annotations

The source gene from the RefSeq v1.1 (IWGSC, 2018) was obtained from ```UK.geneid2CSinformant.txt```, corresponding to the mapping is described in 10+ Wheat Genomes Consortium (2020). Each scaffold was classified according to the corresponding chromosome from the source genes, excluding projections classified as pseudogenes. Then each gene was classified as being consistent or not with the expected scaffold. F


## PGSBv2  annotation

The projected genes where classified as being consistent between the chromosome from the source and projected genes.   

## Both annotations.

Then, the genes on each annotation where filtered to keep the genes that map to the expected chromosome. Finally, the genes that have more than one projection in the expected chromosome where removed.  


In [1]:
require 'zlib'
require 'bio'
require 'csv'
require 'set'
require 'bio-gff3'
require 'daru'
require 'nyaplot'

"if(window['d3'] === undefined ||\n   window['Nyaplot'] === undefined){\n    var path = {\"d3\":\"https://cdnjs.cloudflare.com/ajax/libs/d3/3.5.5/d3.min\",\"downloadable\":\"http://cdn.rawgit.com/domitry/d3-downloadable/master/d3-downloadable\"};\n\n\n\n    var shim = {\"d3\":{\"exports\":\"d3\"},\"downloadable\":{\"exports\":\"downloadable\"}};\n\n    require.config({paths: path, shim:shim});\n\n\nrequire(['d3'], function(d3){window['d3']=d3;console.log('finished loading d3');require(['downloadable'], function(downloadable){window['downloadable']=downloadable;console.log('finished loading downloadable');\n\n\tvar script = d3.select(\"head\")\n\t    .append(\"script\")\n\t    .attr(\"src\", \"http://cdn.rawgit.com/domitry/Nyaplotjs/master/release/nyaplot.js\")\n\t    .attr(\"async\", true);\n\n\tscript[0][0].onload = script[0][0].onreadystatechange = function(){\n\n\n\t    var event = document.createEvent(\"HTMLEvents\");\n\t    event.initEvent(\"load_nyaplot\",false,false);\n\t    win

false

In [2]:
Transcript = Struct.new(:id, :gene, :chromosome,:version,:count,:transcript,:confidence, :count_int, :isoform)
def parseTranscript name
    arr=name.split(".")
    match = /TraesCS(?<chr>[[:alnum:]]{1,2})(?<ver>[[:digit:]]{2})G(?<count>[[:digit:]]+)(?<conf>[[:upper:]]*)/.match arr[0]
    raise "Unable to parse: #{name}" unless match
    Transcript.new(name, arr[0],match[:chr],match[:ver],match[:count],arr[1],match[:conf], match[:count].to_i, arr[1])
end

def parseEITranscript name
    arr=name.split(".")
    match = /Traes(?<chr>[[:upper:]]{3}_scaffold_[[:digit:]]*)_(?<ver>[[:digit:]]{2})G(?<count>[[:digit:]]+)(?<conf>[[:upper:]]*)/.match arr[0]
    raise "Unable to parse: #{name}" unless match
    Transcript.new(name, arr[0],match[:chr].downcase,match[:ver],match[:count],arr[1],match[:conf], match[:count].to_i, arr[1])
end

:parseEITranscript

In [3]:
MrnaStats = Struct.new(:cds_count, :cds_max_gap)
module Bio::GFFbrowser::FastLineParser
  module_function :parse_line_fast
end

class GFF3
    def initialize(file: "", is_gz: true)
        @file = file
        @is_gz = is_gz
    end
    
    def header
        return enum_for(:header) unless block_given? 
        io = nil
        if @is_gz
            infile = open(@file)
            io = Zlib::GzipReader.new(infile) 
        else
            io =  File.open(@file)
        end
        
        io.each_line do |line|  
            line.encode!('UTF-8', 'UTF-8', :invalid => :replace)
            line.strip!
            break if line == '##FASTA'
            break unless line.length == 0 or line =~ /^#/
            yield line
        end
    end
    
    def each
        return enum_for(:each) unless block_given? 
        io = nil
        if @is_gz
            infile = open(@file)
            io = Zlib::GzipReader.new(infile) 
        else
            io =  File.open(@file)
        end
        parser = Bio::GFFbrowser::FastLineParser
        io.each_line do |line|  
            line.encode!('UTF-8', 'UTF-8', :invalid => :replace)
            line.strip!
            break if line == '##FASTA'
            #parents.clear if line == '###'
            next if line.length == 0 or line =~ /^#/
            begin
                record = Bio::GFFbrowser::FastLineRecord.new(parser.parse_line_fast(line))
                yield record
            rescue Exception => e
                $stderr.puts "Unable to parse '#{line}'\n#{e}" 
                throw e
            end
         end
    end
    
    def each_gene
        return enum_for(:each_gene) unless block_given? 
        self.each do |record|
           next unless record.feature == "gene"
            yield record
        end 
    end
    
    def each_mrna
        return enum_for(:each_mrna) unless block_given? 
        self.each do |record|
           next unless record.feature == "mRNA"
            yield record
        end 
    end
    
    def each_cds
        return enum_for(:each_mrna) unless block_given? 
        self.each do |record|
           next unless record.feature == "CDS"
            yield record
        end 
    end
    
    def calculate_mrna_stats
        return if @mrna_stats
        
        @mrna_stats = Hash.new {|h,k| h[k] = MrnaStats.new(0,0) }
        
        last_mrna = ""
        last_record = nil
        each_cds do |record|
            parent = record.get_attribute "Parent"
            mrna = @mrna_stats[parent]
            mrna.cds_count += 1
            
            if last_mrna == parent
                distance =  record.start - last_record.end 
                mrna.cds_max_gap = distance if distance > mrna.cds_max_gap
            end
            last_record = record
            last_mrna   = parent

        end
        return
    end
    
    def mrna_info(id)
        calculate_mrna_stats
        @mrna_stats[id] 
    end
    
    def cds_to_print(mrna,cannonical_exons:[])
        cds_features = [] 
        i = 0
        offset=0
        offset_start=0
        each_cds do |record|
            # Target=Sm1_CDS 3049 3183 +
            target = record.get_attribute "Target"
            arr = target.split(" ")
            col = @@colors[i % @@colors.size ]
            start = arr[1].to_i + offset
            ends = arr[2].to_i + offset
            offset_start = record.start  if offset_start == 0
            tmp = CDS_feature.new(start, ends, col, 
                record.seqid, record.start,record.end, record.start - offset_start )
            #offset = ends
            cds_features << tmp
            i += 1
        end
        cds_features
    end
    
end



    
class MultipleGFFs
    def initialize(folder: "../mapping/", lines:[], suffix:".SM1.cds.sorted.gff", is_gz:false )
        @folder = folder
        @lines = lines
        @suffix = suffix
        @lines_gffs = Hash.new
        @lines.each do |l|
            path ="#{folder}/#{l}#{suffix}"
            @lines_gffs[l] = GFF3.new(file: path, is_gz: is_gz)
            #@lines_gffs[l].calculate_mrna_stats
        end
    end
    
    def method_missing(m, *args, &block)
        @lines_gffs.send(m, *args, &block)
    end
    
    def each_gff
       @lines_gffs.each_pair{|k,v| yield k, v }
    end
    
    def summary
        ret = []
        each_gff do |k,v|
            v.each_mrna do |record|
                #coverage=100.0;identity=100.0;matches=4443;mismatches=0;indels=0;unknowns=0
                tmp = {}
                tmp[:line] = k
                tmp[:id] = record.get_attribute "Name"
                tmp[:chr] = record.seqid
                tmp[:start] = record.start
                tmp[:end] = record.end
                tmp[:strand] = record.strand
                tmp[:genomic_length] = record.end - record.start
                tmp[:coverage] = record.get_attribute "coverage"
                tmp[:identity] = record.get_attribute "identity"
                tmp[:matches]  = record.get_attribute "matches"
                tmp[:mismatches]  = record.get_attribute "mismatches"
                tmp[:indels] = record.get_attribute "indels"
                tmp[:unknowns] = record.get_attribute "unknowns"
                mrna_stats = @lines_gffs[k].mrna_info(record.id)
                tmp[:cds_count]   = mrna_stats.cds_count
                tmp[:cds_max_gap] = mrna_stats.cds_max_gap
                ret << tmp
            end
        end
        ret
    end
    
     attr_reader :lines_gffs
    
    def to_svg(mrna: "Sm1_CDS.mrna1", positions: false, out: nil)
        p = Bio::Graphics::Page.new(width: 800,
             height: 1000, 
              number_of_intervals:10,
            background_color: "white"
            )
        each_gff do |k,v|
            generic_track = p.add_track(:glyph => :generic, 
                            :name => k, 
                            :label => true  )
            v.cds_to_print(mrna).each do |cds|
                
                f_id = positions ? cds.offset_start : nil
                feature = Bio::Graphics::MiniFeature.new(start: cds.start, 
                    end: cds.end,
                    fill_color: cds.color, 
                    id: f_id)
                generic_track.add(feature) 
            end 
        end
        
         
        p.write(out) if out
        
        p.get_markup
    end
end
base_path="/Volumes/PanGenome/References"
lines = ["cadenza","claire", "paragon","robigus","weebil"]
gffs = MultipleGFFs.new(folder: "#{base_path}/releaseV3/gff",
    lines:lines, 
    suffix:".gff.gz", 
    is_gz:true )
puts ""

cad = gffs["cadenza"]

valid_genes = Set.new
gffs.each_gff do |k,v|
    v.each_gene do |g|
        valid_genes << g.id
    end
end

valid_genes.size

610352

This file contains the source genes for the EI assemblies. 

In [4]:
ei_mapping = Daru::DataFrame.from_csv("#{base_path}/releaseV3/gff/UK.geneid2CSinformant.txt", col_sep: "\t")
ei_mapping.size

638111

In [5]:
ei_mapping["chromosome"] = ei_mapping.map(:row) do |row|
  parseTranscript(row["transcript"]).chromosome
end

ei_mapping["gene"] = ei_mapping.map(:row) do |row|
  parseTranscript(row["transcript"]).gene
end
ei_mapping["scaffold"] =  ei_mapping.map(:row) do |row|
    parseEITranscript(row["projected_gene"]).chromosome
end
ei_mapping["is_gene"] =  ei_mapping.map(:row) do |row|
    valid_genes.include? row["projected_gene"]
end
ei_mapping.first 10

,line,projected_gene,transcript,chromosome,gene,scaffold,is_gene
0,cadenza,TraesCAD_scaffold_000001_01G000100,TraesCS1B02G148300.1,1B,TraesCS1B02G148300,cad_scaffold_000001,true
1,cadenza,TraesCAD_scaffold_000001_01G000200,TraesCSU02G134200.1,U,TraesCSU02G134200,cad_scaffold_000001,true
2,cadenza,TraesCAD_scaffold_000001_01G000300,TraesCS7B02G186200.1,7B,TraesCS7B02G186200,cad_scaffold_000001,true
3,cadenza,TraesCAD_scaffold_000003_01G000100,TraesCS6D02G266000.1,6D,TraesCS6D02G266000,cad_scaffold_000003,true
4,cadenza,TraesCAD_scaffold_000003_01G000200,TraesCS6B02G208700.1,6B,TraesCS6B02G208700,cad_scaffold_000003,true
5,cadenza,TraesCAD_scaffold_000003_01G000300,TraesCS6B02G208800.1,6B,TraesCS6B02G208800,cad_scaffold_000003,true
6,cadenza,TraesCAD_scaffold_000004_01G000100,TraesCS7B02G122100.1,7B,TraesCS7B02G122100,cad_scaffold_000004,true
7,cadenza,TraesCAD_scaffold_000004_01G000200,TraesCS6D02G188400.1,6D,TraesCS6D02G188400,cad_scaffold_000004,true
8,cadenza,TraesCAD_scaffold_000004_01G000300,TraesCS5D02G258700.1,5D,TraesCS5D02G258700,cad_scaffold_000004,true
9,cadenza,TraesCAD_scaffold_000004_01G000400,TraesCS6B02G357700.1,6B,TraesCS6B02G357700,cad_scaffold_000004,true


In [6]:
ei_mapping.filter(:row) do |row|
  ["TraesWEE_scaffold_000001_01G000100"].include? row["projected_gene"] 
end

,line,projected_gene,transcript,chromosome,gene,scaffold,is_gene
510301,weebil,TraesWEE_scaffold_000001_01G000100,TraesCS6D02G266000.1,6D,TraesCS6D02G266000,wee_scaffold_000001,true


In [7]:
ei_mapping_sorted = ei_mapping.sort(["scaffold"])
ei_mapping_sorted.size

638111

The following section takes the sorted table by scaffold to get the chromosome consensus of each scaffold. It is based on the majority of the chromosome of the source gene 

In [8]:
i = 0
current_scaffold = ""
current_chromosomes = Hash.new {|h,k| h[k] = 0}
consensus_chromosome = {}
ei_mapping_sorted.each_row do |row|
    if current_scaffold != row["scaffold"]
        if current_chromosomes.size > 0
            max_v = current_chromosomes.max_by{|k,v| v}
            consensus_chromosome[current_scaffold] = [
              current_scaffold, 
              max_v[0], 
              max_v[1],  
              current_chromosomes.values.reduce(:+),
              100 * max_v[1] / current_chromosomes.values.reduce(:+)
            ]
        end
        current_scaffold  = row["scaffold"] 
        current_chromosomes = Hash.new {|h,k| h[k] = 0}
        i+=1
        
    end 
    current_chromosomes[row["chromosome"]] += 1 if row["is_gene"]
    
end
max_v = current_chromosomes.max_by{|k,v| v}

consensus_chromosome[current_scaffold] = [
  current_scaffold, 
  max_v[0],
  max_v[1],
  current_chromosomes.values.reduce(:+), 
  100 * max_v[1] / current_chromosomes.values.reduce(:+)
]

consensus_chr_df = Daru::DataFrame.rows(consensus_chromosome.values , 
    order: [:scaffold, :chr, :hits, :total_genes, :percentage])
consensus_chr_df.head

,scaffold,chr,hits,total_genes,percentage
0,cad_scaffold_000001,1B,1,3,33
1,cad_scaffold_000003,6B,2,3,66
2,cad_scaffold_000004,6B,2,5,40
3,cad_scaffold_000005,7A,1,2,50
4,cad_scaffold_000006,1B,1,2,50
5,cad_scaffold_000007,2B,4,6,66
6,cad_scaffold_000008,7B,5,5,100
7,cad_scaffold_000009,2B,9,10,90
8,cad_scaffold_000010,1B,2,2,100
9,cad_scaffold_000011,7B,1,1,100


In [9]:
consensus_genes_per_scaff = Hash.new {|h,k| h[k] = 0}
genes_per_scaff = Hash.new {|h,k| h[k] = 0}
percentage_cons_per_scaff = Hash.new {|h,k| h[k] = 0}

consensus_chr_df.each_row do |row|
    consensus_genes_per_scaff[  row[:hits]] += 1
    genes_per_scaff[ row[:total_genes]] += 1
    percentage_cons_per_scaff[ 5 * (row[:percentage] / 5 )] += 1
end

percentage_cons_per_scaff

{30=>4156, 65=>12759, 40=>895, 50=>28408, 100=>256295, 90=>313, 80=>4506, 45=>14, 75=>6094, 60=>1879, 25=>731, 70=>624, 55=>398, 85=>1420, 35=>69, 95=>4, 20=>108, 15=>14, 10=>1}

In [10]:
percentage_cons_df = Daru::DataFrame.new({
  percentage: percentage_cons_per_scaff.keys,
  count: percentage_cons_per_scaff.values,
  })
percentage_cons_df.sort!([:percentage])
plot = Nyaplot::Plot.new
bar = plot.add(:line, percentage_cons_df[:percentage].to_a, percentage_cons_df[:count].to_a)
plot.x_label "Percentage of genes corresponding to consensus"
plot.y_label "Count"
plot.show
puts ""

#<Nyaplot::Frame:0x00007fce76335c60 @properties={:panes=>[#<Nyaplot::Plot:0x00007fce7632c408 @properties={:diagrams=>[#<Nyaplot::Diagram:0x00007fce76336318 @properties={:type=>:line, :options=>{:x=>"data0", :y=>"data1"}, :data=>"1c3e99c5-88a4-4c96-afcc-341ab367dd9e"}, @xrange=[10, 100], @yrange=[1, 256295]>], :options=>{:x_label=>"Percentage of genes corresponding to consensus", :y_label=>"Count", :zoom=>true, :width=>700, :xrange=>[10, 100], :yrange=>[1, 256295]}}>], :data=>{"1c3e99c5-88a4-4c96-afcc-341ab367dd9e"=>#<Nyaplot::DataFrame:0x00007fce76337a88 @name="1c3e99c5-88a4-4c96-afcc-341ab367dd9e", @rows=[{:data0=>10, :data1=>1}, {:data0=>15, :data1=>14}, {:data0=>20, :data1=>108}, {:data0=>25, :data1=>731}, {:data0=>30, :data1=>4156}, {:data0=>35, :data1=>69}, {:data0=>40, :data1=>895}, {:data0=>45, :data1=>14}, {:data0=>50, :data1=>28408}, {:data0=>55, :data1=>398}, {:data0=>60, :data1=>1879}, {:data0=>65, :data1=>12759}, {:data0=>70, :data1=>624}, {:data0=>75, :data1=>6094}, {:data0=>80, :data1=>4506}, {:data0=>85, :data1=>1420}, {:data0=>90, :data1=>313}, {:data0=>95, :data1=>4}, {:data0=>100, :data1=>256295}]>}, :extension=>[]}>

In [11]:
consensus_chr_df.write_csv("./EI_consensus_genes.csv")

In [12]:
cons_hash = {}
consensus_chr_df.each_row{|row| cons_hash[row[:scaffold]] = row[:chr]}
cons_hash.first


["cad_scaffold_000001", "1B"]

Comparing each consensus chromosome to the gene.

In [13]:
ei_mapping["is_chr_gene_consistent"] =  ei_mapping.map(:row) do |row|
    cons_hash[row["scaffold"]] == row["chromosome"]
end
ei_mapping.first 20

,line,projected_gene,transcript,chromosome,gene,scaffold,is_gene,is_chr_gene_consistent
0,cadenza,TraesCAD_scaffold_000001_01G000100,TraesCS1B02G148300.1,1B,TraesCS1B02G148300,cad_scaffold_000001,true,true
1,cadenza,TraesCAD_scaffold_000001_01G000200,TraesCSU02G134200.1,U,TraesCSU02G134200,cad_scaffold_000001,true,false
2,cadenza,TraesCAD_scaffold_000001_01G000300,TraesCS7B02G186200.1,7B,TraesCS7B02G186200,cad_scaffold_000001,true,false
3,cadenza,TraesCAD_scaffold_000003_01G000100,TraesCS6D02G266000.1,6D,TraesCS6D02G266000,cad_scaffold_000003,true,false
4,cadenza,TraesCAD_scaffold_000003_01G000200,TraesCS6B02G208700.1,6B,TraesCS6B02G208700,cad_scaffold_000003,true,true
5,cadenza,TraesCAD_scaffold_000003_01G000300,TraesCS6B02G208800.1,6B,TraesCS6B02G208800,cad_scaffold_000003,true,true
6,cadenza,TraesCAD_scaffold_000004_01G000100,TraesCS7B02G122100.1,7B,TraesCS7B02G122100,cad_scaffold_000004,true,false
7,cadenza,TraesCAD_scaffold_000004_01G000200,TraesCS6D02G188400.1,6D,TraesCS6D02G188400,cad_scaffold_000004,true,false
8,cadenza,TraesCAD_scaffold_000004_01G000300,TraesCS5D02G258700.1,5D,TraesCS5D02G258700,cad_scaffold_000004,true,false
9,cadenza,TraesCAD_scaffold_000004_01G000400,TraesCS6B02G357700.1,6B,TraesCS6B02G357700,cad_scaffold_000004,true,true


Comparing that it is not a pseudogene and that it is consistent. 

In [14]:
gene_count = Hash.new{|h,k| h[k] = 0}
ei_mapping.each_row do |row|
    next unless row["is_gene"] and row["is_chr_gene_consistent"]
    gene_count[row["line"] + "_" + row["gene"]] += 1
end
puts gene_count.size

455268


Finally, comparing that the gene is mapped only once (Removes posible dupplications or genes that are ambigous)

In [15]:
ei_mapping["is_unique"] =  ei_mapping.map(:row) do |row|
    gene_count[row["line"] + "_" + row["gene"]] == 1 and  row["is_gene"] and row["is_chr_gene_consistent"] 
end
ei_mapping.head

,line,projected_gene,transcript,chromosome,gene,scaffold,is_gene,is_chr_gene_consistent,is_unique
0,cadenza,TraesCAD_scaffold_000001_01G000100,TraesCS1B02G148300.1,1B,TraesCS1B02G148300,cad_scaffold_000001,true,true,false
1,cadenza,TraesCAD_scaffold_000001_01G000200,TraesCSU02G134200.1,U,TraesCSU02G134200,cad_scaffold_000001,true,false,false
2,cadenza,TraesCAD_scaffold_000001_01G000300,TraesCS7B02G186200.1,7B,TraesCS7B02G186200,cad_scaffold_000001,true,false,false
3,cadenza,TraesCAD_scaffold_000003_01G000100,TraesCS6D02G266000.1,6D,TraesCS6D02G266000,cad_scaffold_000003,true,false,false
4,cadenza,TraesCAD_scaffold_000003_01G000200,TraesCS6B02G208700.1,6B,TraesCS6B02G208700,cad_scaffold_000003,true,true,false
5,cadenza,TraesCAD_scaffold_000003_01G000300,TraesCS6B02G208800.1,6B,TraesCS6B02G208800,cad_scaffold_000003,true,true,true
6,cadenza,TraesCAD_scaffold_000004_01G000100,TraesCS7B02G122100.1,7B,TraesCS7B02G122100,cad_scaffold_000004,true,false,false
7,cadenza,TraesCAD_scaffold_000004_01G000200,TraesCS6D02G188400.1,6D,TraesCS6D02G188400,cad_scaffold_000004,true,false,false
8,cadenza,TraesCAD_scaffold_000004_01G000300,TraesCS5D02G258700.1,5D,TraesCS5D02G258700,cad_scaffold_000004,true,false,false
9,cadenza,TraesCAD_scaffold_000004_01G000400,TraesCS6B02G357700.1,6B,TraesCS6B02G357700,cad_scaffold_000004,true,true,true


In [16]:
consensus_chr_df.write_csv("EI_consensus_genes.csv")
ei_mapping.write_csv("ei_mapping.csv")

In [17]:
ei_mapping["line"].uniq

,line
0,cadenza
128544,claire
255850,paragon
383153,robigus
510301,weebil


# PGSB Assemblies 

In [18]:
lines = [
    "arinalrfor",
    "chinese",
    "jagger",
    "julius",
    "lancer",
    "landmark",
    "mace",
    "norin61",
    "spelta",
    "stanley",
    "sy_mattis"
    ]
gffs_pgsb = MultipleGFFs.new(folder: "#{base_path}/releasePGSBv2.0/gff",
    lines:lines, 
    suffix:".gff.gz", 
    is_gz:true )
puts ""

In [19]:
def parsePGSBTranscript name
    arr=name.split(".")
    match = /Traes(?<variety>[[:upper:]]{3})(?<chr>[[:alnum:]]{1,2})(?<ver>[[:digit:]]{2})G(?<count>[[:digit:]]+)(?<conf>[[:upper:]]*)/.match arr[0]
    #match  = /Traes(?<variety>[[:upper:]]*)/.match arr[0]
    #puts match.inspect
    raise "Unable to parse: #{name}" unless match
    Transcript.new(name, arr[0],match[:chr],match[:ver],match[:count],match[:variety], match[:conf],match[:count].to_i, arr[1])
end
rows = []
gffs_pgsb.each_gff do |k,v|
    v.each_gene do |record|
        srcmodel = record.attributes["srcmodel"]
        parsed_proj = parsePGSBTranscript record.id
        parsed_src = parseTranscript srcmodel
        
        rows << [k, 
            record.id, 
            record.attributes["srcmodel"],
            parsed_src.chromosome, 
            parsed_src.gene,
            parsed_proj.chromosome,  
            record.attributes["disruptedORF"].nil?, 
            parsed_proj.chromosome == parsed_src.chromosome ]      
    end
end

pgsb_df = Daru::DataFrame.rows(rows , 
    order: ["line", "projected_gene", "transcript", "chromosome", "gene", "scaffold", "is_gene", "is_chr_gene_consistent"])
pgsb_df.head

,line,projected_gene,transcript,chromosome,gene,scaffold,is_gene,is_chr_gene_consistent
0,arinalrfor,TraesARI1A01G000100,TraesCS1A02G003546.1,1A,TraesCS1A02G003546,1A,false,true
1,arinalrfor,TraesARI1A01G000200,TraesCS1A02G004126.1,1A,TraesCS1A02G004126,1A,true,true
2,arinalrfor,TraesARI1A01G000300,TraesCS1A02G003600.1,1A,TraesCS1A02G003600,1A,true,true
3,arinalrfor,TraesARI1A01G000400,TraesCS1A02G003700.1,1A,TraesCS1A02G003700,1A,true,true
4,arinalrfor,TraesARI1A01G000500,TraesCS7A02G485500.1,7A,TraesCS7A02G485500,1A,true,false
5,arinalrfor,TraesARI1A01G000600,TraesCS3A02G080300.1,3A,TraesCS3A02G080300,1A,true,false
6,arinalrfor,TraesARI1A01G000700,TraesCS6D02G375000.1,6D,TraesCS6D02G375000,1A,true,false
7,arinalrfor,TraesARI1A01G000800,TraesCSU02G097400.1,U,TraesCSU02G097400,1A,true,false
8,arinalrfor,TraesARI1A01G000900,TraesCS1A02G019300.1,1A,TraesCS1A02G019300,1A,true,true
9,arinalrfor,TraesARI1A01G001000,TraesCS1A02G003500.1,1A,TraesCS1A02G003500,1A,true,true


Validating that the chromosome from the source gene is consistent with the chromosome. 

In [20]:
gene_count_pgsb = Hash.new{|h,k| h[k] = 0}
pgsb_df.each_row do |row|
    next unless row["is_gene"] and row["is_chr_gene_consistent"]
    gene_count_pgsb[row["line"] + "_" + row["gene"]] += 1
end
puts gene_count_pgsb.size

1069393


In [21]:
pgsb_df["is_unique"] =  pgsb_df.map(:row) do |row|
    gene_count_pgsb[row["line"] + "_" + row["gene"]] == 1 and  row["is_gene"] and row["is_chr_gene_consistent"] 
end
pgsb_df.head

,line,projected_gene,transcript,chromosome,gene,scaffold,is_gene,is_chr_gene_consistent,is_unique
0,arinalrfor,TraesARI1A01G000100,TraesCS1A02G003546.1,1A,TraesCS1A02G003546,1A,false,true,false
1,arinalrfor,TraesARI1A01G000200,TraesCS1A02G004126.1,1A,TraesCS1A02G004126,1A,true,true,true
2,arinalrfor,TraesARI1A01G000300,TraesCS1A02G003600.1,1A,TraesCS1A02G003600,1A,true,true,true
3,arinalrfor,TraesARI1A01G000400,TraesCS1A02G003700.1,1A,TraesCS1A02G003700,1A,true,true,true
4,arinalrfor,TraesARI1A01G000500,TraesCS7A02G485500.1,7A,TraesCS7A02G485500,1A,true,false,false
5,arinalrfor,TraesARI1A01G000600,TraesCS3A02G080300.1,3A,TraesCS3A02G080300,1A,true,false,false
6,arinalrfor,TraesARI1A01G000700,TraesCS6D02G375000.1,6D,TraesCS6D02G375000,1A,true,false,false
7,arinalrfor,TraesARI1A01G000800,TraesCSU02G097400.1,U,TraesCSU02G097400,1A,true,false,false
8,arinalrfor,TraesARI1A01G000900,TraesCS1A02G019300.1,1A,TraesCS1A02G019300,1A,true,true,true
9,arinalrfor,TraesARI1A01G001000,TraesCS1A02G003500.1,1A,TraesCS1A02G003500,1A,true,true,true


In [22]:
pgsb_df.write_csv("pgsb_mapping.csv")
all_genes = pgsb_df.concat ei_mapping
all_genes.write_csv("all_mapping.csv")
filtered_and_sorted = all_genes.where(all_genes["is_unique"]).sort(["gene","line"])
filtered_and_sorted.write_csv("sorted_filtered_mapping.csv")
filtered_and_sorted.head

,line,projected_gene,transcript,chromosome,gene,scaffold,is_gene,is_chr_gene_consistent,is_unique
244046,jagger,TraesJAG1A01G000500,TraesCS1A02G000100.1,1A,TraesCS1A02G000100,1A,true,true,true
363868,julius,TraesJUL1A01G036600,TraesCS1A02G000100.1,1A,TraesCS1A02G000100,1A,true,true,true
482640,lancer,TraesLAC1A01G000300,TraesCS1A02G000100.1,1A,TraesCS1A02G000100,1A,true,true,true
602686,landmark,TraesLDM1A01G000400,TraesCS1A02G000100.1,1A,TraesCS1A02G000100,1A,true,true,true
721733,mace,TraesMAC1A01G002400,TraesCS1A02G000100.1,1A,TraesCS1A02G000100,1A,true,true,true
841525,norin61,TraesNOR1A01G004400,TraesCS1A02G000100.1,1A,TraesCS1A02G000100,1A,true,true,true
960218,spelta,TraesTSP1A01G000300,TraesCS1A02G000100.1,1A,TraesCS1A02G000100,1A,true,true,true
1080407,stanley,TraesSTA1A01G008800,TraesCS1A02G000100.1,1A,TraesCS1A02G000100,1A,true,true,true
1448033,cadenza,TraesCAD_scaffold_628837_01G000200,TraesCS1A02G000200.1,1A,TraesCS1A02G000200,cad_scaffold_628837,true,true,true
1569964,claire,TraesCLE_scaffold_182945_01G000100,TraesCS1A02G000200.1,1A,TraesCS1A02G000200,cle_scaffold_182945,true,true,true
